<a href="https://colab.research.google.com/github/delphython/de_final_task_5/blob/main/de_final_task_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.6 MB/s eta 0:00:00


In [2]:
!pip install pyspark

In [3]:
import csv
from faker import Faker
import random

fake = Faker()

num_records = 100000

http_methods = ['GET', 'POST', 'PUT', 'DELETE']
response_codes = [200, 301, 404, 500]

file_path = "/content/task/web_server_logs.csv"

with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ip', 'timestamp', 'method', 'url', 'response_code', 'response_size'])

    for _ in range(num_records):
        ip = fake.ipv4()
        timestamp = fake.date_time_this_year().isoformat()
        method = random.choice(http_methods)
        url = fake.uri_path()
        response_code = random.choice(response_codes)
        response_size = random.randint(100, 10000)

        writer.writerow([ip, timestamp, method, url, response_code, response_size])

print(f"Сгенерировано {num_records} записей и сохранено в {file_path}")


Сгенерировано 100000 записей и сохранено в /content/task/web_server_logs.csv


In [12]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder \
.appName("DE fifth final task") \
.getOrCreate()

web_server_logs_df = spark_session.read.csv(
    "/content/task/web_server_logs.csv",
    header=True,
    inferSchema=True,
)

web_server_logs_df.createOrReplaceTempView("web_server_logs")

top_ten_active_ip_addresses = spark_session.sql("""
select ip, count(ip) as request_count
from web_server_logs
group by ip
order by request_count desc
LIMIT 10
""")
print("Top 10 active ip addresses:")
top_ten_active_ip_addresses.show()

request_count_by_method = spark_session.sql("""
select method, count(ip) as method_count
from web_server_logs
group by method
""")
print("Request count by HTTP method:")
request_count_by_method.show()

not_found_response_code_count = spark_session.sql("""
select count(ip)
from web_server_logs
where response_code = '404'
""").collect()[0][0]
print(f"Number of 404 response codes: {not_found_response_code_count}")

total_response_size_by_day = spark_session.sql("""
select date(timestamp) as date, sum(response_size) as total_response_size
from web_server_logs
group by date
order by date
""")
print("Total response size by day:")
total_response_size_by_day.show()

spark_session.stop()

Top 10 active ip addresses:
+--------------+-------------+
|            ip|request_count|
+--------------+-------------+
|   81.2.97.253|            2|
|  67.21.79.225|            1|
|152.215.85.170|            1|
|  185.5.113.64|            1|
|   77.79.62.18|            1|
|154.212.17.194|            1|
|  175.76.37.31|            1|
|  38.234.233.2|            1|
|69.131.167.195|            1|
| 82.212.142.51|            1|
+--------------+-------------+

Request count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|  POST|       25208|
|DELETE|       25123|
|   PUT|       24841|
|   GET|       24828|
+------+------------+

Number of 404 response codes: 25023
Total response size by day:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-01-01|            1451089|
|2024-01-02|            1385931|
|2024-01-03|            1500077|
|2024-01-04|            1272725|
|2024-01-05|            1378954|
|2